In [6]:
from selenium import webdriver
import time
import pandas as pd
import tqdm
from tqdm.notebook import tqdm

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import array_contains, udf
from pymongo import MongoClient

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("lang=ko_KR") # 한국어!
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36")
path='/home/ubuntu/chromedriver'

driver = webdriver.Chrome(path, chrome_options=chrome_options) # 드라이버 경로 설정
url_list = [] # 블로그 url을 저장하기 위한 변수
title_list=[]
results=[]
content_list = "" # 블로그 content를 누적하기 위한 변수
text = "감성숙소" # 검색어
 
for i in range(1, 3):  # 1~X-1페이지까지의 블로그 내용을 읽어옴 573
    url = 'https://section.blog.naver.com/Search/Post.nhn?pageNo='+ str(i) + '&rangeType=ALL&orderBy=sim&keyword=' + text # url 값 설정
    driver.get(url)
    time.sleep(1) # 오류 방지 sleep
 
    for j in range(1, 8):
        blog_urls = driver.find_element_by_xpath('/html/body/ui-view/div/main/div/div/section/div[2]/div['+str(j)+']/div/div[1]/div[1]/a[1]')
        blog_url = blog_urls.get_attribute('href')
        url_list.append(blog_url)
        
        title = driver.find_element_by_xpath('//*[@id="content"]/section/div[2]/div['+str(j)+']/div/div[1]/div[1]/a[1]/strong/span').text
        title_list.append(title)
        


df= pd.DataFrame({'url':url_list,'title':title_list})
df.to_csv("blog_url.csv", encoding='utf-8-sig')
  

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:29: DeprecationWarning: use options instead of chrome_options


In [7]:
  
    
# url 리스트 불러오기    
url_load = pd.read_csv("blog_url.csv")

num_list = len(url_load)
        #data=[blog_url, title]
        #results.append(data)
dict = {} # 전체 크롤링 데이터를 담을 그릇

for k in tqdm(range(0, num_list)): # 저장했던 블로그 하나씩 순회
    # 글 띄우기
    url=url_load['url'][k]
    driver = webdriver.Chrome(path, chrome_options=chrome_options)
    driver.get(url)
    
    try:
        # iframe 접근
        driver.switch_to.frame('mainFrame')
        target_info={}
   
#         overlays1=".se-fs-.se-ff-"
#         tit = driver.find_element_by_css_selector(overlays1)
#         title = tit.text
        
        # 글쓴이 크롤링
        overlays2=".nick"
        nick = driver.find_element_by_css_selector(overlays2)
        nickname = nick.text
     
        # 날짜 크롤링
        overlays3=".se_publishDate.pcol2"
        date = driver.find_element_by_css_selector(overlays3)
        datetime = date.text  
        
        # 내용 크롤링
        overlays4 = ".se-component.se-text.se-l-default" # 내용 크롤링
        contents = driver.find_elements_by_css_selector(overlays4)

        
        content_list=[]
        for content in contents:
            content_list.append(content.text) # 각 블로그의 내용을 변수에 누적함
            #content_list = content_list.replace("\n","") #공백 제거
        content_str= ' '.join(content_list)
#     for content in contents:
#         content_list = content_list + content.text # 각 블로그의 내용을 변수에 누적함
#         content_list = content_list.replace("\n","") #공백 제거

        # 글 하나는 target_info 딕셔너리에 담고
        target_info['title']=title_list[k]
        target_info['nickname']=nickname
        target_info['datetime']=datetime
        target_info['content']=content_str

        
        # 각각의 글은 dict 딕셔너리에 담는다
        dict[k]=target_info
        time.sleep(1)
        
        # 블로그 하나 크롤링 후 크롬 창 닫기
        driver.close()
    except:
        driver.close()
        time.sleep(1)
        continue

result_df = pd.DataFrame.from_dict(dict, 'index')
result_df.to_csv("naver.csv", encoding='utf-8-sig')

f=pd.read_csv("naver.csv", encoding="utf-8")
keep_col = ["title","nickname","datetime","content"]
new_f = f[keep_col].replace('\\n',' ', regex=True)
new_f.to_csv("naver_blog_crawling.csv", encoding="utf-8-sig", index=False)

f=pd.read_csv("naver_blog_crawling.csv")
    
# data=[url_list, title_list, content_list]

  0%|          | 0/14 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: use options instead of chrome_options


0      2021. 9. 8. 15:05
1      2021. 9. 4. 23:34
2      2021. 9. 13. 0:27
3     2021. 8. 22. 12:55
4     2021. 8. 17. 23:56
5      2021. 9. 1. 18:14
6     2021. 8. 18. 10:13
7                 21시간 전
8      2021. 9. 11. 0:07
9     2021. 8. 23. 15:17
10    2021. 8. 25. 22:53
11     2021. 9. 6. 13:55
12     2021. 7. 4. 22:29
13     2021. 7. 3. 16:29
Name: datetime, dtype: object